In [1]:

# import
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings

from langchain import LLMChain
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.llms.base import LLM
from transformers import AutoTokenizer, AutoModel, AutoConfig
from typing import Any, Dict, List, Mapping, Optional, Tuple, Union
from torch.mps import empty_cache
import torch
from langchain.chains import RetrievalQA


/home/dxw22/anaconda3/envs/agentllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class GLM(LLM):
    max_token: int = 2048
    temperature: float = 0.8
    top_p = 0.9
    tokenizer: object = None
    model: object = None
    history_len: int = 1024
    
    def __init__(self):
        super().__init__()
        
    @property
    def _llm_type(self) -> str:
        return "GLM"
            
    def load_model(self, llm_device="gpu",model_name_or_path=None):
        model_config = AutoConfig.from_pretrained(model_name_or_path, trust_remote_code=True)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,trust_remote_code=True)
        self.model = AutoModel.from_pretrained(model_name_or_path, config=model_config, trust_remote_code=True, device='cuda:5').half() # GLM模块装在gpu: 6



    def _call(self,prompt:str,history:List[str] = [],stop: Optional[List[str]] = None):
        response, _ = self.model.chat(
                    self.tokenizer,prompt,
                    history=history[-self.history_len:] if self.history_len > 0 else [],
                    max_length=self.max_token,temperature=self.temperature,
                    top_p=self.top_p)
        return response

In [3]:
# 访谈记录的原始文本加载
loader = TextLoader("/data1/dxw_data/code/MKT/GLM-mkt/GLM_RAG_MKT/data/Interview.txt", encoding="utf-8")
documents = loader.load()


In [4]:
# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# create the open-source embedding function
model_kwargs = {'device': 'cuda:6'}  # embedding模块装在gpu: 7
embedding_function = HuggingFaceEmbeddings(model_name='/data1/dxw_data/code/MKT/GLM-mkt/text2vec-large-chinese',model_kwargs=model_kwargs) # 会报错“No sentence-transformers model found”但是不影响使用,这只是huggingface的检测问题。

# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)



Created a chunk of size 285, which is longer than the specified 100
Created a chunk of size 125, which is longer than the specified 100
Created a chunk of size 367, which is longer than the specified 100
Created a chunk of size 168, which is longer than the specified 100
Created a chunk of size 229, which is longer than the specified 100
Created a chunk of size 247, which is longer than the specified 100
Created a chunk of size 220, which is longer than the specified 100
Created a chunk of size 133, which is longer than the specified 100
Created a chunk of size 157, which is longer than the specified 100
Created a chunk of size 101, which is longer than the specified 100
Created a chunk of size 208, which is longer than the specified 100
Created a chunk of size 120, which is longer than the specified 100
Created a chunk of size 191, which is longer than the specified 100
Created a chunk of size 165, which is longer than the specified 100
Created a chunk of size 102, which is longer tha

In [5]:
import sys
modelpath = "/data1/dxw_data/code/Agent/code/ChatGLM3/chatglm3-6b"
sys.path.append(modelpath)
llm = GLM()
llm.load_model(model_name_or_path = modelpath)
#---------------------------至此, 成功加载模型

Loading checkpoint shards: 100%|██████████| 7/7 [00:11<00:00,  1.71s/it]


In [6]:
# ----------------------------开始总结数据------------------------ #

In [8]:
# 直接概括访谈纪要文本的内容，但效果不好，会有遗漏
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())
query = "根据文本的提问,概括出文本内容？" # 根据业务逻辑
qa.run(query)

'文本内容是关于公司提供高端解决方案和设备控制技术的访谈记录。公司强调自己的技术优势在于打标速度快，基于不同的控制办法。镜片定制主要是因为影响温漂和反射率等因素。材料外采有一定要求，主要是对镀膜工艺有特别要求。大功率切割的硬件套与激光振镜相比，控制部分技术难度相近，但打标软件技术积累时间较长，靠解决工艺问题积累。'

In [9]:
# print("文本内容是关于公司提供高端解决方案和设备控制技术的访谈记录。公司强调自己的技术优势在于打标速度快，基于不同的控制办法。镜片定制主要是因为影响温漂和反射率等因素。材料外采有一定要求，主要是对镀膜工艺有特别要求。大功率切割的硬件套与激光振镜相比，控制部分技术难度相近，但打标软件技术积累时间较长，靠解决工艺问题积累。")


文本内容是关于公司提供高端解决方案和设备控制技术的访谈记录。公司强调自己的技术优势在于打标速度快，基于不同的控制办法。镜片定制主要是因为影响温漂和反射率等因素。材料外采有一定要求，主要是对镀膜工艺有特别要求。大功率切割的硬件套与激光振镜相比，控制部分技术难度相近，但打标软件技术积累时间较长，靠解决工艺问题积累。


In [ ]:
# ----------------------------尝试多步sequence-chain总结------------------------ #



In [10]:
# ---------------------------------------------------- #
 # 业务逻辑: 
    # 1) 每次提取访谈记录的文本
    # 2) 然后根据问题先提炼主题，过滤杂质。
    # 3) 通过根据问题得到的主题来归纳文本，防止盲目归纳文本出现幻觉。
# ok手动实现多个链条。注意不是简单的LLMChain提问，而是每次都和db交互的向量数据库方案链条
chainone = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())
query="根据文本的提问,概括出文本有哪些主题?直接回复结果"
resultone = chainone({"query": query})
theme=resultone["result"]
print("提炼的主题: ",theme) #回答result["result"]
print("提炼的主题:  文本的主题包括：振镜水平的高低判断，镜片定制，材料来源，大功率切割硬件套与激光振镜的比较，打标软件的技术难度，以及企业技术优势和目的。")
 # ---------------------------------------------------- #
chaintwo = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())
query=f"“{theme}”,\n\n根据上面的主题,请用几句话描述和精简文本的内容？"
resulttwo = chaintwo({"query": query})
# print("问题: ",resulttwo["query"])
print("总结的结果: ",resulttwo["result"])
print("总结的结果:  文本主要讨论了振镜水平判断、镜片定制、材料来源、大功率切割硬件套与激光振镜的比较、打标软件的技术难度以及企业技术优势和目的。其中，振镜水平判断的方法主要是价格。镜片定制主要是因为影响温漂等因素。材料来源方面，有些是外采的，有些则需自行制作。大功率切割硬件套与激光振镜的比较主要在于控制部分的技术难度。打标软件的技术难度在于解决工艺问题积累。企业技术优势和目的是在精密加工高端领域与振镜厂商合作定制控制卡等。")
 


 

/home/dxw22/anaconda3/envs/agentllm/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


提炼的主题:  文本的主题包括：振镜水平的高低判断，镜片定制，材料来源，大功率切割硬件套与激光振镜的比较，打标软件的技术难度，以及企业技术优势和目的。
总结的结果:  文本主要讨论了振镜水平判断、镜片定制、材料来源、大功率切割硬件套与激光振镜的比较、打标软件的技术难度以及企业技术优势和目的。其中，振镜水平判断的方法主要是价格。镜片定制主要是因为影响温漂等因素。材料来源方面，有些是外采的，有些则需自行制作。大功率切割硬件套与激光振镜的比较主要在于控制部分的技术难度。打标软件的技术难度在于解决工艺问题积累。企业技术优势和目的是在精密加工高端领域与振镜厂商合作定制控制卡等。
